In [3]:
import pandas as pd
from sqlalchemy import create_engine

# Database connection string
engine = create_engine('postgresql://uvg_user:uvg_password@db:5432/health_data')

# Simple query to test the foundation layer
try:
    df = pd.read_sql("SELECT 1 as connection_status", engine)
    print("Connection Successful! Your Biomedical Data Stack is ready.")
    print(df)
except Exception as e:
    print(f"Connection Failed: {e}")

Connection Successful! Your Biomedical Data Stack is ready.
   connection_status
0                  1


In [6]:
from sqlalchemy import text

with engine.connect() as conn:
    result = conn.execute(text("SELECT 1;"))
    print(result.fetchone())


(1,)


In [16]:
pd.read_sql(
        text("""
            SELECT subject_id, external_id, full_name, sex, date_of_birth
            FROM patients
            WHERE external_id LIKE 'MRN-9%'
            ORDER BY external_id;
        """),
        engine
    )

,subject_id,external_id,full_name,sex,date_of_birth
0,1,MRN-9001,Paciente Sin Fecha,F,None
1,2,MRN-9002,None,M,1990-01-01
2,3,MRN-9003,Paciente Ciudad Typo,M,1985-09-10


In [13]:
query = """
SELECT COUNT(*) AS n_missing_dob
FROM patients
WHERE date_of_birth IS NULL;
"""
pd.read_sql(query, engine)

,n_missing_dob
0,1


In [17]:
query = """
SELECT COUNT(*) AS n_missing_name
FROM patients
WHERE full_name IS NULL;
"""
pd.read_sql(query, engine)

,n_missing_name
0,1


In [20]:
from sqlalchemy import text

with engine.connect() as conn:
    pd.read_sql(
        text("""
        SELECT COUNT(*) AS n_typos_guate
        FROM diagnoses
        WHERE diagnosis_text LIKE '%Guate%';
        """),
        conn
    )
    df = pd.read_sql(text(query), conn)

df


,n_typos_guate
0,1
